# Battle of Neighbourhoods



## Capstone Project by IBM

## Objective
In this project I am going to determine how NEW YORK CITY and TORONTO are similar or dissimilar by comparing the most common venue categories in their neighborhoods.
## Background
New York and Toronto, two major cities of the world. Lots of businesses and companies have their head offices in and around these cities, which urges lots of people from around the world from different proffesional background to come and settle here. Additionally, many people have to move among these two cities too, i.e., New York to Toronto or vice versa for job requirements or business needs.
Suppose if a resident of New York has to permanently move to Toronto and he/she is searching for a place to live there. That person would like to move to similar area as their current area in New York based on nearby venues including coffee shops or schools or parks or types of restaurants, etc.
And also for new comers as both cities are very diverse and are the financial capitals of their respective countries it will be helpful for someone wanting to choose between these two places for having an ideal place to live where he or she can compare things like commutiny culture, good educational institution, hangout places etc.

# A description of the data and how it will be used to solve the problem.

Here I will use the **Foursquare API** to get the location data to explore neighborhoods in New York City and Toronto. I will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the  **k-means clustering algorithm** to complete this task. Finally, I will use the **Folium library** to visualize the neighborhoods in New York City and Toronto city and their emerging clusters.
<br>
Along with this other libraries like **Sklearn, Pandas, Numpy, Matplotlib** will be used for the ananlysis, clustering algorithm , easy data manipulation and data visualization work.
<br>
Firstly I will collect the datas for all the neighborhood areas in NEW YORK AND TORONTO with the help of the URLs shared with us in this course , then I will collect the latitude and longitude datas for all those neighborhoods , here also we have access to those datas in this course, then I will combine these two dataframes as we need only one clustering to 
understand the similarity between these two major cities.
<br>
In this combined data frame, I will apply K-means clustering algorithm to group similar venues and then compare each venue category for both cities.

## Importing useful libraries

In [1]:
import numpy as np
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 102kB 11.5MB/s ta 0:00:01
Libraries imported.


# Methodology

## Data cleaning

### New York Data

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

### Transform data into pandas dataframe

In [5]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [8]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Use geopy library to get the latitude and longitude values of New York City.

In [10]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create a map of New York with neighborhoods superimposed on top.

In [11]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [13]:
manhattan_data['City'] = 'New York'

In [14]:
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Manhattan,Marble Hill,40.876551,-73.910660,New York
1,Manhattan,Chinatown,40.715618,-73.994279,New York
2,Manhattan,Washington Heights,40.851903,-73.936900,New York
3,Manhattan,Inwood,40.867684,-73.921210,New York
4,Manhattan,Hamilton Heights,40.823604,-73.949688,New York


## Toronto Data

reading the data table from web link given

In [15]:
d=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [16]:
len(d)

3

taking the data in a dataframe

In [17]:
df=d[0]
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


drop the cells with a borough that is Not assigned.

In [18]:
df.replace("Not assigned", np.nan, inplace = True)

In [19]:
df.dropna(subset=["Borough"], axis=0, inplace=True)

df.reset_index(drop=True, inplace=True)

In [20]:
df.Neighborhood.fillna(df.Borough, inplace=True)

In [21]:
df.head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [22]:
df_1=df.groupby('Postal Code')['Borough','Neighborhood'].agg(','.join).reset_index()

In [23]:
df_1.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Get co ordinates for the postal code
<br>
Here we will read the location datas from the csv file given

In [24]:
df_2=pd.read_csv('http://cocl.us/Geospatial_data')

In [25]:
df_2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
df_final=pd.concat([df_1, df_2], axis=1, join='inner', sort=False)

In [27]:
df_final = df_final.loc[:,~df_final.columns.duplicated()]
df_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [28]:
toronto_data=df_final.drop('Postal Code',1)
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [29]:
toronto_data['City'] = 'Toronto'
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude,City
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Toronto
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Toronto
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Toronto
3,Scarborough,Woburn,43.770992,-79.216917,Toronto
4,Scarborough,Cedarbrae,43.773136,-79.239476,Toronto


### Merging the data of new york city and toronto

In [30]:
data_final=manhattan_data.append(toronto_data, sort=False)

In [31]:
data_final.reset_index()
#data_final.drop('index',1)
data_final.shape

(143, 5)

In [159]:
s=pd.Series(range(0,143))
data_final=data_final.set_index([s])
data_final

,Borough,Neighborhood,Latitude,Longitude,City
0,Manhattan,Marble Hill,40.876551,-73.910660,New York
1,Manhattan,Chinatown,40.715618,-73.994279,New York
2,Manhattan,Washington Heights,40.851903,-73.936900,New York
3,Manhattan,Inwood,40.867684,-73.921210,New York
4,Manhattan,Hamilton Heights,40.823604,-73.949688,New York
5,Manhattan,Manhattanville,40.816934,-73.957385,New York
6,Manhattan,Central Harlem,40.815976,-73.943211,New York
7,Manhattan,East Harlem,40.792249,-73.944182,New York
8,Manhattan,Upper East Side,40.775639,-73.960508,New York
9,Manhattan,Yorkville,40.775930,-73.947118,New York


## Define Foursquare Credentials and Version

In [32]:
CLIENT_ID = 'YXTN4COOQI2NXGZOZONR44CSTBYQIP2DFOHBP2FK4NIPV5MJ' # your Foursquare ID
CLIENT_SECRET = 'OZKMMJXN53TCHOQOBEC2S5KV35IV1JMCB0TSC0JKXLDGZVSL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YXTN4COOQI2NXGZOZONR44CSTBYQIP2DFOHBP2FK4NIPV5MJ
CLIENT_SECRET:OZKMMJXN53TCHOQOBEC2S5KV35IV1JMCB0TSC0JKXLDGZVSL


borrow the **get_category_type** function from the Foursquare lab.

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Let's create a function to get the top 100 venues in all the neighborhoods in New York and Toronto

In [34]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Now write the code to run the above function on each neighborhood and create a new dataframe called Newyork&Toronto-venues.

In [35]:
Newyork_Toronto_venues = getNearbyVenues(names=data_final['Neighborhood'],
                                   latitudes=data_final['Latitude'],
                                   longitudes=data_final['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux Wes

## Let's check the size of the resulting dataframe

In [36]:
print(Newyork_Toronto_venues.shape)
Newyork_Toronto_venues.head()

(5235, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


Let's check how many venues were returned for each neighborhood

In [37]:
Newyork_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Battery Park City,66,66,66,66,66,66
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22


## Let's find out how many unique categories can be curated from all the returned venues

In [38]:
print('There are {} uniques categories.'.format(len(Newyork_Toronto_venues['Venue Category'].unique())))

There are 391 uniques categories.


## Analyze Each Neighborhood

In [39]:
Newyork_Toronto_onehot = pd.get_dummies(Newyork_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Newyork_Toronto_onehot['Neighborhood'] = Newyork_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Newyork_Toronto_onehot.columns[-1]] + list(Newyork_Toronto_onehot.columns[:-1])
Newyork_Toronto_onehot = Newyork_Toronto_onehot[fixed_columns]

Newyork_Toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Out

In [40]:
Newyork_Toronto_onehot.shape

(5235, 391)

## Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
Newyork_Toronto_grouped = Newyork_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Newyork_Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Out

In [42]:
Newyork_Toronto_grouped.shape

(136, 391)

## Let's print each neighborhood along with the top 5 most common venues

In [43]:
num_top_venues = 5

for hood in Newyork_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Newyork_Toronto_grouped[Newyork_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2             Clothing Store   0.2
3               Skating Rink   0.2
4             Breakfast Spot   0.2


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1           Pub  0.11
2          Pool  0.11
3  Skating Rink  0.11
4           Gym  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2  Middle Eastern Restaurant  0.05
3              Deli / Bodega  0.05
4              Shopping Mall  0.05


----Battery Park City----
           venue  freq
0           Park  0.12
1          Hotel  0.06
2    Coffee Shop  0.06
3            Gym  0.05
4  Memorial Site  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3   Chinese Restaurant  0.25
4 

                       venue  freq
0             Clothing Store  0.09
1                Coffee Shop  0.08
2        Japanese Restaurant  0.03
3         Italian Restaurant  0.03
4  Middle Eastern Restaurant  0.03


----Glencairn----
                 venue  freq
0  Japanese Restaurant  0.25
1                  Pub  0.25
2                 Park  0.25
3        Metro Station  0.25
4          Yoga Studio  0.00


----Golden Mile, Clairlea, Oakridge----
            venue  freq
0        Bus Line   0.2
1          Bakery   0.2
2    Soccer Field   0.1
3  Ice Cream Shop   0.1
4            Park   0.1


----Gramercy----
                venue  freq
0          Bagel Shop  0.06
1                 Bar  0.05
2         Coffee Shop  0.05
3         Pizza Place  0.05
4  Italian Restaurant  0.03


----Greenwich Village----
                venue  freq
0  Italian Restaurant  0.08
1                Café  0.05
2    Sushi Restaurant  0.04
3         Pizza Place  0.03
4                 Gym  0.03


----Guildwood, Morningsid

            venue  freq
0  Clothing Store  0.10
1     Coffee Shop  0.10
2     Yoga Studio  0.05
3      Restaurant  0.05
4             Spa  0.05


----Northwest, West Humber - Clairville----
                 venue  freq
0  Rental Car Location  0.33
1                  Bar  0.33
2            Drugstore  0.33
3          Yoga Studio  0.00
4            Nightclub  0.00


----Northwood Park, York University----
                  venue  freq
0        Massage Studio  0.25
1                   Bar  0.25
2  Caribbean Restaurant  0.25
3           Coffee Shop  0.25
4           Yoga Studio  0.00


----Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East----
                   venue  freq
0         Baseball Field   1.0
1            Yoga Studio   0.0
2  Outdoors & Recreation   0.0
3   Other Great Outdoors   0.0
4        Organic Grocery   0.0


----Parkdale, Roncesvalles----
            venue  freq
0  Breakfast Spot  0.14
1 

                       venue  freq
0                Pizza Place  0.25
1         Chinese Restaurant  0.12
2  Middle Eastern Restaurant  0.12
3               Intersection  0.12
4             Discount Store  0.12


----Weston----
                   venue  freq
0      Convenience Store   0.5
1                   Park   0.5
2  Outdoors & Recreation   0.0
3   Other Great Outdoors   0.0
4        Organic Grocery   0.0


----Wexford, Maryvale----
                       venue  freq
0                     Bakery  0.25
1  Middle Eastern Restaurant  0.25
2                 Smoke Shop  0.25
3             Breakfast Spot  0.25
4                Yoga Studio  0.00


----Willowdale, Willowdale East----
              venue  freq
0  Ramen Restaurant  0.09
1              Café  0.06
2       Pizza Place  0.06
3  Sushi Restaurant  0.06
4        Restaurant  0.06


----Willowdale, Willowdale West----
           venue  freq
0       Pharmacy   0.2
1           Bank   0.2
2  Grocery Store   0.2
3    Pizza Place   0.2
4 

## Let's put that into a pandas dataframe

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = Newyork_Toronto_grouped['Neighborhood']

for ind in np.arange(Newyork_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(Newyork_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Clothing Store,Skating Rink,Lounge,Empanada Restaurant,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
1,"Alderwood, Long Branch",Pizza Place,Pool,Coffee Shop,Pub,Sandwich Place,Skating Rink,Pharmacy,Gym,Dog Run,Doner Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Gas Station,Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Pizza Place,Supermarket,Sushi Restaurant,Shopping Mall
3,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Gym,Playground,Beer Garden,Shopping Mall,Plaza,Food Court
4,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Empanada Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
5,"Bedford Park, Lawrence Manor East",Restaurant,Coffee Shop,Sandwich Place,Italian Restaurant,Comfort Food Restaurant,Grocery Store,Pharmacy,Pub,Cupcake Shop,Café
6,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Seafood Restaurant,Pub,Cheese Shop,Beer Bar,Restaurant,Lounge
7,"Birch Cliff, Cliffside West",General Entertainment,College Stadium,Skating Rink,Café,Women's Store,Empanada Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop
8,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Climbing Gym,Intersection,Bar,Furniture / Home Store,Bakery,Nightclub,Grocery Store
9,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Pizza Place


# Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [46]:
kclusters = 8

Newyork_Toronto_grouped_clustering = Newyork_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans1 = KMeans(n_clusters=kclusters, random_state=0).fit(Newyork_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans1.labels_[0:10] 

array([5, 0, 5, 5, 5, 5, 5, 5, 5, 5], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [47]:
neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans1.labels_)

Newyork_Toronto_merged = data_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Newyork_Toronto_merged = Newyork_Toronto_merged.join(neighborhoods_venues_sorted1.set_index('Neighborhood'), on='Neighborhood')

Newyork_Toronto_merged # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,New York,5.0,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Miscellaneous Shop,Steakhouse,Shopping Mall,Supplement Shop,Seafood Restaurant,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,New York,5.0,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Vietnamese Restaurant,American Restaurant,Bar,Coffee Shop,Spa,Optical Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,New York,5.0,Café,Bakery,Spanish Restaurant,Mobile Phone Shop,Grocery Store,Chinese Restaurant,Supermarket,Sandwich Place,Bank,Tapas Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,New York,5.0,Mexican Restaurant,Restaurant,Café,Lounge,Chinese Restaurant,Bakery,American Restaurant,Caribbean Restaurant,Spanish Restaurant,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,New York,5.0,Pizza Place,Coffee Shop,Deli / Bodega,Café,Mexican Restaurant,School,Bakery,Cocktail Bar,Sushi Restaurant,Chinese Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,New York,5.0,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Park,Chinese Restaurant,Bank,Bus Station,Gastropub,Climbing Gym
6,Manhattan,Central Harlem,40.815976,-73.943211,New York,5.0,African Restaurant,Seafood Restaurant,Cosmetics Shop,French Restaurant,Chinese Restaurant,American Restaurant,Bar,Boutique,Jazz Club,BBQ Joint
7,Manhattan,East Harlem,40.792249,-73.944182,New York,5.0,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Deli / Bodega,Pharmacy,Donut Shop,Doctor's Office,Sandwich Place,French Restaurant
8,Manhattan,Upper East Side,40.775639,-73.960508,New York,5.0,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,Juice Bar,Spa,French Restaurant,Exhibit,Yoga Studio,Wine Shop
9,Manhattan,Yorkville,40.775930,-73.947118,New York,5.0,Coffee Shop,Gym,Italian Restaurant,Bar,Sushi Restaurant,Deli / Bodega,Pizza Place,Mexican Restaurant,Diner,Japanese Restaurant


In [48]:
Newyork_Toronto_merged.dropna(subset = ['Cluster Labels'], inplace=True)

In [49]:
#Newyork_Toronto_merged['Cluster Labels'].astype(int)
cols = ['Cluster Labels']
Newyork_Toronto_merged[cols] = Newyork_Toronto_merged[cols].applymap(np.int64)
#Newyork_Toronto_merged.Cluster Labels = Newyork_Toronto_merged.Cluster Labels.astype(int)

Finally, let's visualize the resulting clusters

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Newyork_Toronto_merged['Latitude'], Newyork_Toronto_merged['Longitude'], Newyork_Toronto_merged['Neighborhood'], Newyork_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

### Cluster 1

In [181]:
Newyork_Toronto_merged.loc[Newyork_Toronto_merged['Cluster Labels'] == 0, Newyork_Toronto_merged.columns[[1] + list(range(4, Newyork_Toronto_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Scarborough Village,Toronto,0,Playground,Pizza Place,Women's Store,English Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
53,"Clarks Corners, Tam O'Shanter, Sullivan",Toronto,0,Pharmacy,Pizza Place,Convenience Store,Fast Food Restaurant,Fried Chicken Joint,Noodle House,Gas Station,Bank,Chinese Restaurant,Italian Restaurant
64,"Willowdale, Willowdale West",Toronto,0,Bank,Pizza Place,Grocery Store,Pharmacy,Coffee Shop,Women's Store,Electronics Store,Donut Shop,Drugstore,Dry Cleaner
75,"Parkview Hill, Woodbine Gardens",Toronto,0,Pizza Place,Bank,Pharmacy,Athletics & Sports,Intersection,Pet Store,Gym / Fitness Center,Fast Food Restaurant,Gastropub,Dry Cleaner
121,"Runnymede, The Junction North",Toronto,0,Convenience Store,Pizza Place,Grocery Store,Bus Line,Women's Store,Empanada Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant
129,"Alderwood, Long Branch",Toronto,0,Pizza Place,Pool,Coffee Shop,Pub,Sandwich Place,Skating Rink,Pharmacy,Gym,Dog Run,Doner Restaurant
136,Humber Summit,Toronto,0,Pizza Place,Furniture / Home Store,Women's Store,English Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
139,Westmount,Toronto,0,Pizza Place,Middle Eastern Restaurant,Chinese Restaurant,Discount Store,Sandwich Place,Intersection,Coffee Shop,Women's Store,Electronics Store,Drugstore
141,"South Steeles, Silverstone, Humbergate, Jamest...",Toronto,0,Grocery Store,Beer Store,Fast Food Restaurant,Pizza Place,Sandwich Place,Pharmacy,Fried Chicken Joint,Electronics Store,Donut Shop,Drugstore


### Cluster 2

In [182]:
Newyork_Toronto_merged.loc[Newyork_Toronto_merged['Cluster Labels'] == 1, Newyork_Toronto_merged.columns[[1] + list(range(4, Newyork_Toronto_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,"Golden Mile, Clairlea, Oakridge",Toronto,1,Bakery,Bus Line,Ice Cream Shop,Park,Intersection,Bus Station,Soccer Field,Metro Station,Farmers Market,Empanada Restaurant
51,"Wexford, Maryvale",Toronto,1,Middle Eastern Restaurant,Breakfast Spot,Bakery,Smoke Shop,Ethiopian Restaurant,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant
84,Lawrence Park,Toronto,1,Bus Line,Park,Swim School,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant
119,"North Park, Maple Leaf Park, Upwood Park",Toronto,1,Construction & Landscaping,Bakery,Park,Basketball Court,English Restaurant,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant
134,"West Deane Park, Princess Gardens, Martin Grov...",Toronto,1,Filipino Restaurant,Women's Store,English Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant


### Cluster 3

In [183]:
Newyork_Toronto_merged.loc[Newyork_Toronto_merged['Cluster Labels'] == 2, Newyork_Toronto_merged.columns[[1] + list(range(4, Newyork_Toronto_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"Rouge Hill, Port Union, Highland Creek",Toronto,2,Bar,Women's Store,Doner Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
142,"Northwest, West Humber - Clairville",Toronto,2,Drugstore,Rental Car Location,Bar,Women's Store,English Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant


### Cluster 4

In [184]:
Newyork_Toronto_merged.loc[Newyork_Toronto_merged['Cluster Labels'] == 3, Newyork_Toronto_merged.columns[[1] + list(range(4, Newyork_Toronto_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
40,"Malvern, Rouge",Toronto,3,Fast Food Restaurant,Women's Store,English Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant


### Cluster 5

In [185]:
Newyork_Toronto_merged.loc[Newyork_Toronto_merged['Cluster Labels'] == 4, Newyork_Toronto_merged.columns[[1] + list(range(4, Newyork_Toronto_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,"York Mills, Silver Hills",Toronto,4,Cafeteria,Women's Store,Doner Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


### Cluster 6

In [186]:
Newyork_Toronto_merged.loc[Newyork_Toronto_merged['Cluster Labels'] == 5, Newyork_Toronto_merged.columns[[1] + list(range(4, Newyork_Toronto_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,New York,5,Sandwich Place,Gym,Coffee Shop,Yoga Studio,Miscellaneous Shop,Steakhouse,Shopping Mall,Supplement Shop,Seafood Restaurant,Donut Shop
1,Chinatown,New York,5,Chinese Restaurant,Bakery,Cocktail Bar,Bubble Tea Shop,Vietnamese Restaurant,American Restaurant,Bar,Coffee Shop,Spa,Optical Shop
2,Washington Heights,New York,5,Café,Bakery,Spanish Restaurant,Mobile Phone Shop,Grocery Store,Chinese Restaurant,Supermarket,Sandwich Place,Bank,Tapas Restaurant
3,Inwood,New York,5,Mexican Restaurant,Restaurant,Café,Lounge,Chinese Restaurant,Bakery,American Restaurant,Caribbean Restaurant,Spanish Restaurant,Deli / Bodega
4,Hamilton Heights,New York,5,Pizza Place,Coffee Shop,Deli / Bodega,Café,Mexican Restaurant,School,Bakery,Cocktail Bar,Sushi Restaurant,Chinese Restaurant
5,Manhattanville,New York,5,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Park,Chinese Restaurant,Bank,Bus Station,Gastropub,Climbing Gym
6,Central Harlem,New York,5,African Restaurant,Seafood Restaurant,Cosmetics Shop,French Restaurant,Chinese Restaurant,American Restaurant,Bar,Boutique,Jazz Club,BBQ Joint
7,East Harlem,New York,5,Mexican Restaurant,Bakery,Thai Restaurant,Latin American Restaurant,Deli / Bodega,Pharmacy,Donut Shop,Doctor's Office,Sandwich Place,French Restaurant
8,Upper East Side,New York,5,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,Juice Bar,Spa,French Restaurant,Exhibit,Yoga Studio,Wine Shop
9,Yorkville,New York,5,Coffee Shop,Gym,Italian Restaurant,Bar,Sushi Restaurant,Deli / Bodega,Pizza Place,Mexican Restaurant,Diner,Japanese Restaurant


### Cluster 7

In [187]:
Newyork_Toronto_merged.loc[Newyork_Toronto_merged['Cluster Labels'] == 6, Newyork_Toronto_merged.columns[[1] + list(range(4, Newyork_Toronto_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
131,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Toronto,6,Baseball Field,Women's Store,Doner Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
137,"Humberlea, Emery",Toronto,6,Food Service,Baseball Field,Women's Store,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant


### Cluster 8

In [188]:
Newyork_Toronto_merged.loc[Newyork_Toronto_merged['Cluster Labels'] == 7, Newyork_Toronto_merged.columns[[1] + list(range(4, Newyork_Toronto_merged.shape[1]))]]

,Neighborhood,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,"Milliken, Agincourt North, Steeles East, L'Amo...",Toronto,7,Playground,Park,Coffee Shop,Women's Store,Empanada Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop
63,York Mills West,Toronto,7,Convenience Store,Park,Women's Store,English Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
65,Parkwoods,Toronto,7,Food & Drink Shop,Park,Women's Store,English Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant
80,"East Toronto, Broadview North (Old East York)",Toronto,7,Park,Convenience Store,Pizza Place,Coffee Shop,Empanada Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop
88,"Moore Park, Summerhill East",Toronto,7,Summer Camp,Park,Restaurant,Women's Store,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant
90,Rosedale,Toronto,7,Park,Playground,Trail,Women's Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant
104,"Forest Hill North & West, Forest Hill Road Park",Toronto,7,Jewelry Store,Park,Sushi Restaurant,Trail,Women's Store,Empanada Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant
113,Humewood-Cedarvale,Toronto,7,Hockey Arena,Field,Park,Trail,Women's Store,Empanada Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant
114,Caledonia-Fairbanks,Toronto,7,Park,Women's Store,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant
138,Weston,Toronto,7,Convenience Store,Park,Women's Store,English Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant


## Results and Discussion

As we can see here all New York / Manhattan neighbourhoods were clustered into only one group, although by increasing the number of clusters in the Kmeans algorithm we will find different clusters in the New York neighbourhoods too. Here for K=8 , cluster 6 include all the neighbourhoods in New York and some neighbourhoods from Toronto. The 1st common venues in this cluster neighbourhoods are cafes, fine dinning continental restaurants, grocery stores etc. **So if someone wanting to switch places between New York and Toronto ,they should look for places in the neighbourhoods from cluster 6 to get a similar feelings with their current location after shifting .**
<br>
And also people who are searching for some kind of events in these two city's, they can narrow down their search area by looking at these 8 different clusters depending on their interest. If they are looking for a Pizza place, they should look for neighbourhoods in cluster 1. Cluster 2 neighbourhoods have mostly breakfast places, bakery shops etc. For drugstore , women's store cluster 3 neighbourhoods are ideal. Cluster 4, cluster 5 and cluster 7 have very less number of neighbourhoods , so these places are outliers in clustering distribution. Cluster 6 includes the most happening places in these two citys , here one can find amazing nightlifes with excellents chinese, italian, mexican restaurants and bars, nice coffee shops, supermarkets etc. Cluster 8 mainly group places with playground, many kind of sports arena ,parks etc.

## Conclusion

The objective of this project was to find similar neighbourhoods in New York and Toronto by applying K-means clustering algorithm in their neighbourhoods and grouping then based on some discriminationg category. From the  visuation of the result  we can see that we weren't able to cluster places within New York, this is because of very large number of input data and less number of the assumed K value in this algorithm. Although we were able to find the most similar places in these two cities, that is neghbouhoods from cluster 6.